In [3]:
import numpy as np
import pylab as pb
import GPy
import copy as cp

pb.ion()

file_base = "modularized_v4_backup/"

X = np.load("hc_baby.npy", allow_pickle=True)
Y = np.log(np.load("samples_baby.npy", allow_pickle=True))

In [5]:
X_raw = np.load(file_base + "hc_massive_predictor.npy", allow_pickle=True)
Y_raw = np.log(np.load(file_base + "samples_massive_predictor.npy",
    allow_pickle=True))

In [7]:
# We have to truncate the matrices a little because they use the shapes of the
# final products, meaning a lot of unfilled cells
X_truncated = X_raw#[:backup_index]
Y_truncated = Y_raw#[:backup_index]
# Possible off-by-one error here, but in this case that just means we're
# excluding one sample.

In [8]:
scale_factor = np.load("standard_k.npy", allow_pickle=True)

In [9]:
# This plot only serves to make sure that the first row actually describes a
# function.
pb.plot(scale_factor, Y_truncated[0])

In [10]:
'''
for i in range(len(Y_truncated)):
    pspec = Y_truncated[i]
    if float('-inf') in pspec:
        print("-inf")
    if float('inf') in pspec:
        print("inf")
    if None in pspec:
        print("None")
    if True in np.isnan(pspec):
        print("NaN in row", i)
        print(X_truncated[i])
    if 0 in pspec:
        print("zero")
'''

'\nfor i in range(len(Y_truncated)):\n    pspec = Y_truncated[i]\n    if float(\'-inf\') in pspec:\n        print("-inf")\n    if float(\'inf\') in pspec:\n        print("inf")\n    if None in pspec:\n        print("None")\n    if True in np.isnan(pspec):\n        print("NaN in row", i)\n        print(X_truncated[i])\n    if 0 in pspec:\n        print("zero")\n'

In [11]:
def is_bad(row):
    if float('-inf') in row or float('inf') in row \
        or None in row or True in np.isnan(pspec) \
        or 0 in row:
        return True
    return False

bad_row_indices = []
for i in range(len(Y_truncated)):
    pspec = Y_truncated[i]
    if is_bad(pspec):
        bad_row_indices.append(i)

X = np.delete(X_truncated, bad_row_indices, 0)
Y = np.delete(Y_truncated, bad_row_indices, 0)

'''
# Now remove the offending rows
for i in reversed(bad_row_indices):
    # iterate backwards or else concurrent modification
    # makes the indices meaningless
    X.pop(i)
    Y.pop(i)
'''

'\n# Now remove the offending rows\nfor i in reversed(bad_row_indices):\n    # iterate backwards or else concurrent modification\n    # makes the indices meaningless\n    X.pop(i)\n    Y.pop(i)\n'

In [12]:
# Now the goal is to compute summary statistics on Y for each value k
len(Y[0])
len(Y[:, 0])
mu = np.mean(Y, axis=0)
mu.shape
stdev = np.std(Y, axis=0)
stdev.shape

(300,)

In [13]:
# now normalize the data with these summary statistics
Y_normalized = cp.deepcopy(Y)

for i in range(len(Y)):
    Y_normalized[i] -= mu
    Y_normalized[i] /= stdev

In [14]:
def inverse_transform(raw_prediction):
    return np.exp(raw_prediction * stdev + mu)

In [15]:
print(X.shape, Y_normalized.shape)

(4987, 6) (4987, 300)


In [16]:
kernel = GPy.kern.RBF(input_dim=6, variance=1., lengthscale=1.)
m = GPy.models.GPRegression(X,Y_normalized,kernel)

In [ ]:
m.constrain_positive('') # '' is a regex matching all parameter names
m.optimize()

In [ ]:
# Remember that the structure of a hc entry is
# omb, omc, ns, s12, omnu, As
x = np.array([np.array([
    0.022445, 0.120567, 0.96, 0.78667255, 0.002, 2.12723788013000E-09
])])
x

In [ ]:
guess, uncertainties = m.predict(x)

In [ ]:
#import matplotlib.pyplot as plt
pb.loglog(scale_factor, inverse_transform(guess[0]))
pb.title("Emulator F, 5000 Samples, Aletheia model 0")
pb.ylabel('P(k) at redshift zero')
pb.xlabel('scale factor k')

In [ ]:
# Now let's check our work with battery
import importlib.util
import sys
spec = importlib.util.spec_from_file_location(
    "spectra", "../shared/spectra.py")
spectra = importlib.util.module_from_spec(spec)
sys.modules["spectra"] = spectra
spec.loader.exec_module(spectra)

# Compare like to like
massless_cosmology = spectra.specify_neutrino_mass(spectra.cosm.iloc[0], 0.002,
    1)
k_m0truth, z_m0truth, p_m0truth, s_m0truth = \
    spectra.kzps(massless_cosmology, zs=[0], k_points=300)
print(s_m0truth)

Next steps:
* Import "ground truth": what the plot was really supposed to look like for model 0
* Play with omega_nu, n_s, omega_nu + omega_cdm to see if you can get the same behavior as in the animations

In [ ]:
#import matplotlib.pyplot as plt
pb.loglog(k_m0truth, p_m0truth)
pb.title("CAMB, Aletheia model 0")
pb.ylabel('P(k) at redshift zero')
pb.xlabel('scale factor k')

In [ ]:
pb.plot(scale_factor, inverse_transform(guess[0]),
    label="F5000 emulator")
pb.loglog(k_m0truth, p_m0truth, label="CAMB")
pb.title("Aletheia model 0, redshift zero")
pb.ylabel('$P(k)$ [Mpc$^3$]')
pb.xlabel('$k$ [1 / Mpc]')
pb.legend()

In [ ]:
pb.plot(scale_factor,
    (inverse_transform(guess[0]) - p_m0truth) / p_m0truth * 100)
pb.xscale('log')
pb.title("Aletheia model 0, redshift zero, percent error")
pb.ylabel('% Error in $P(k)$')
pb.xlabel('$k$ [1 / Mpc]')

Pretty encouraging for such an early iteration.

In [ ]:
# Remember that the structure of a hc entry is
# omb, omc, ns, s12, omnu, As
x_hi_ns = np.array([np.array([
    0.022445, 0.120567, 1.1, 0.82466774, 0, 2.12723788013000E-09
])])
guess_hi_ns, uncertainties_hi_ns = m.predict(x_hi_ns)
x_lo_ns = np.array([np.array([
    0.022445, 0.120567, 0.84, 0.82466774, 0, 2.12723788013000E-09
])])
guess_lo_ns, uncertainties_lo_ns = m.predict(x_lo_ns)


In [ ]:
pb.plot(scale_factor, guess[0], label="n_s 0.96")
pb.plot(scale_factor, guess_hi_ns[0], label="n_s 1.1")
pb.plot(scale_factor, guess_lo_ns[0], label="n_s 0.84")

pb.xscale('log')

pb.title("Aletheia model 0, redshift zero")
pb.ylabel('log $P(k)$')
pb.xlabel('$k$')
pb.legend()

In [ ]:
# Remember that the structure of a hc entry is
# omb, omc, ns, s12, omnu, As
omnu_hi = 0.01
x_hi_omnu = np.array([np.array([
    0.022445, 0.120567 - omnu_hi, 0.96, 0.82466774, omnu_hi,\
    2.12723788013000E-09
])])
guess_hi_omnu, uncertainties_hi_omnu = m.predict(x_hi_omnu)

omnu_mid = 0.002148659574468
x_mid_omnu = np.array([np.array([
    0.022445, 0.120567 - omnu_mid, 0.96, 0.82466774, omnu_mid, \
    2.12723788013000E-09
])])
guess_mid_omnu, uncertainties_mid_omnu = m.predict(x_mid_omnu)


In [ ]:
pb.plot(scale_factor, guess[0], label="omnu 0")
pb.plot(scale_factor, guess_hi_omnu[0], label="omnu 0.01")
pb.plot(scale_factor, guess_mid_omnu[0], label="omnu 0.002")

pb.xscale('log')

pb.title("Aletheia model 0, redshift zero")
pb.ylabel('log $P(k)$')
pb.xlabel('$k$')
pb.legend()

In [ ]:
# Remember that the structure of a hc entry is
# omb, omc, ns, s12, omnu, As
hi_factor = 1.15
x_hi_omm = np.array([np.array([
    0.022445 * hi_factor, 0.120567 * hi_factor, 0.96, 0.82466774, 0,\
    2.12723788013000E-09
])])
guess_hi_omm, uncertainties_hi_omm = m.predict(x_hi_omm)

lo_factor = 0.85
x_lo_omm = np.array([np.array([
    0.022445 * lo_factor, 0.120567 * lo_factor, 0.96, 0.82466774, 0, \
    2.12723788013000E-09
])])
guess_lo_omm, uncertainties_lo_omm = m.predict(x_lo_omm)


In [ ]:
pb.plot(scale_factor, guess[0], label="omm Planck")
pb.plot(scale_factor, guess_hi_omm[0], label="omm Planck * 1.15")
pb.plot(scale_factor, guess_lo_omm[0], label="omm Planck * 0.85")

pb.xscale('log')

pb.title("Aletheia model 0, redshift zero")
pb.ylabel('log $P(k)$')
pb.xlabel('$k$')
pb.legend()

In [ ]:
# Remember that the structure of a hc entry is
# omb, omc, ns, s12, omnu, As
x_hi_As = np.array([np.array([
    0.022445, 0.120567, 0.96, 0.82466774, 0, np.exp(3.91) / 10 ** 10
])])
guess_hi_As, uncertainties_hi_As = m.predict(x_hi_As)
x_lo_As = np.array([np.array([
    0.022445, 0.120567, 0.96, 0.82466774, 0, np.exp(2.35) / 10 ** 10 
])])
guess_lo_As, uncertainties_lo_As = m.predict(x_lo_As)

In [ ]:
pb.plot(scale_factor, guess[0], label="As Planck")
pb.plot(scale_factor, guess_hi_As[0], label="As high")
pb.plot(scale_factor, guess_lo_As[0], label="As low")

pb.xscale('log')

pb.title("Aletheia model 0, redshift zero")
pb.ylabel('log $P(k)$')
pb.xlabel('$k$')
pb.legend()

I guess this shows that sigma12 countermands As, huh?